In [1]:
!pip install --upgrade gensim

     |████████████████████████████████| 24.1 MB 11.6 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [2]:
#mounting google drive
from google.colab import drive 
drive.mount('/content/gdrive')
df = pd.read_csv('/content/gdrive/MyDrive/DATA_690_NLP_Project_Recommender_System/Ujjwal/new_datasets/reviews_7.csv')

Mounted at /content/gdrive


NameError: ignored

In [ ]:
#downloading model to calculate similarity matrix on the basis of the downloaded model
import gensim.downloader as api
model = api.load('word2vec-google-news-300')

In [ ]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import plotly.express as px
from gensim.similarities import SparseTermSimilarityMatrix, WordEmbeddingSimilarityIndex

#grouping by the description on basis of game names
ProductReviewSummary = df.groupby("app_name")["Description"].apply(str)

p = ProductReviewSummary.to_frame()

#the below code will generate matrix that will help in finding soft cosine similarity matrix on basis of description
p['Description'] = p['Description'].str.split(' ')
documents1 = p['Description']

from gensim.corpora import Dictionary
dictionary1 = Dictionary(documents1)

sentence1 = []
for i in range(0, len(p)):
  sentence1.append(dictionary1.doc2bow(p['Description'][i]))

from gensim.models import TfidfModel
documents2 = sentence1
tfidf1 = TfidfModel(documents2)

for i in range(0, len(sentence1)):
  sentence1[i] = tfidf1[sentence1[i]]

termsim_index1 = WordEmbeddingSimilarityIndex(model)
termsim_matrix1 = SparseTermSimilarityMatrix(termsim_index1, dictionary1, tfidf1)

#grouping by the reviews on basis of game names
ProductReviewSummary = df.groupby("app_name")["clean_review_text"].apply(str)

p = ProductReviewSummary.to_frame()

#the below code will generate matrix that will help in finding soft cosine similarity matrix on basis of reviews
p['clean_review_text'] = p['clean_review_text'].str.split(' ')
documents2 = p['clean_review_text']

from gensim.corpora import Dictionary
dictionary2 = Dictionary(documents2)

sentence2 = []
for i in range(0, len(p)):
  sentence2.append(dictionary2.doc2bow(p['clean_review_text'][i]))

from gensim.models import TfidfModel
documents2 = sentence2
tfidf2 = TfidfModel(documents2)

for i in range(0, len(sentence2)):
  sentence2[i] = tfidf2[sentence2[i]]

termsim_index2 = WordEmbeddingSimilarityIndex(model)
termsim_matrix2 = SparseTermSimilarityMatrix(termsim_index2, dictionary2, tfidf2)


In [ ]:
#saving the soft cosine similarity matrices based on Description and Reviews on drive (These matrices will be dynamically added in the application runtime to generate soft cosine similarity of a particular game on basis of Description as well as Reviews)
termsim_matrix1.save('DescMatrix.trm')
termsim_matrix2.save('RevMatrix.trm')